# DreamPlace Algorithms

This notebook tries to reproduce some of the functionality of
the DreamPlace paper

There there are three important algorithms:

* Global Placement (GP)
* Legalization (LG)
* Detailed Placement (DP)

OBS: Implementation below only talk about x direction. The same can be achieve for the y direction.

### Notations:

* V : set of cells
* P : set of pins
* E : set of nets
* B : set of bins


* $x_e^+$ : $max_{i \in e} x_i, \forall e \in E$
* $x_e^-$ : $min_{i \in e} x_i, \forall e \in E$


* $a_i^+$ : $e^\frac{x_i-x_e^+}{\gamma}, \forall i\in e, e \in E$
* $a_i^-$ : $e^\frac{x_i-x_e^-}{\gamma}, \forall i\in e, e \in E$


* $b_e^+$ : $\sum_{i \in e}  a^+, \forall e \in E$
* $b_e^-$ : $\sum_{i \in e}  a^-, \forall e \in E$


* $c_e^+$ : $\sum_{i \in e} x_i a^+, \forall e \in E$
* $c_e^-$ : $\sum_{i \in e} x_i a^-, \forall e \in E$


* $\vec x^+$ : $\{x_e^+\}, \forall e \in E$
* $\vec x^-$ : $\{x_e^-\}, \forall e \in E$


* $\vec a^+$ : $\{a_e^+\}, \forall e \in P$
* $\vec a^-$ : $\{a_e^-\}, \forall i \in P$


* $\vec b^+$ : $\{b_e^+\}, \forall e \in E$
* $\vec b^-$ : $\{b_e^-\}, \forall e \in E$


* $\vec c^+$ : $\{c_e^+\}, \forall e \in E$
* $\vec c^-$ : $\{c_e^-\}, \forall e \in E$

### Process

With the variables below, the GP problem becomes:

* $WL$ - Function that calculates Wirelength cost function
  * Takes any net $e$ and returns wirelenght
* $D$ -  Regularization function (density penalty to spread out cells)

$min(\sum_{e\in E} WL(e;\vec{x},\vec{y} +\alpha D(\vec{x},\vec{y})$

In [2]:
# TODO

Actual used formula: Weighted-average wirelength

$WA_e=\frac{\sum_{i \in e} x_i e^{\frac{x_i}{\gamma}}}{\sum_{i \in e} e^{\frac{x_i}{\gamma}}}$
$-\frac{\sum_{i \in e} x_i e^{\frac{-x_i}{\gamma}}}{\sum_{i \in e} e^{\frac{-x_i}{\gamma}}}$


With:

* $\gamma$ - parameter to control the smoothness and accuracy of the approximation to half-permieter wirelenght (HPWL)

In [3]:
# TODO

### Steps:

Initial placement: Standard cells are placed in the center of the layout with a small Gaussian noise.
The scales of noise are set to 0.1% of the width and height of the placement region in the experiment.


GP: GlobalPlacement Iterations: Loop that involves the computation of wirelength and density gradient, optimization engines, and cell location updating.

LG: Legalization is performed to remove remaining overlaps and align cells to placements sites

DP: Detailed placement to refine the placement solutions using NTUplace3.

In [4]:
# TODO